In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
!git clone git@github.com:swaggy66/M-ABSA.git

Cloning into 'M-ABSA'...


In [2]:
import os
import pandas as pd
import datasets

from glob import glob
from tqdm.auto import tqdm
from konoise import NoiseGenerator

In [23]:
genertor = NoiseGenerator()
noise_methods = ['change-vowels', 'palatalization', 'linking', 'liquidization', 'nasalization', 'assimilation']
categories = []
splits = []
sentences = []
sentences_noisy = []
noise_method_selected = []
for path in tqdm(glob('M-ABSA/data/*/ko/*.txt')):
    # 경로를 os.path.normpath로 정리하고, 폴더 이름 분리
    parts = os.path.normpath(path).split(os.sep)
    category = parts[2]  # 3번째 폴더
    split = os.path.splitext(parts[-1])[0]  # 파일명 (확장자 제외)
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for sentence in tqdm(lines):
            sentence = sentence.strip()
            sentence = sentence.split('####')[0]
            for noise_method in noise_methods:
                sentence_noisy = genertor.generate(sentence, noise_method, 1)[0][0]
                if sentence == sentence_noisy:
                    continue
                categories.append(category)
                splits.append(split)
                sentences.append(sentence)   
                sentences_noisy.append(sentence_noisy)
                noise_method_selected.append(noise_method)
            

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/336 [00:00<?, ?it/s]

  0%|          | 0/566 [00:00<?, ?it/s]

  0%|          | 0/1278 [00:00<?, ?it/s]

  0%|          | 0/312 [00:00<?, ?it/s]

...

  0%|          | 0/544 [00:00<?, ?it/s]

  0%|          | 0/1264 [00:00<?, ?it/s]

  0%|          | 0/281 [00:00<?, ?it/s]

  0%|          | 0/524 [00:00<?, ?it/s]

  0%|          | 0/1181 [00:00<?, ?it/s]

In [24]:
df = pd.DataFrame({
    'category': categories,
    'split': splits,
    'sentence': sentences,
    'sentence_noisy': sentences_noisy,
    'noise_method': noise_method_selected
})

In [25]:
train_df = df[df['split'] == 'train'].drop(columns=['split']).reset_index(drop=True)
groupby = train_df.groupby(['sentence'])[['noise_method']].count().reset_index()
not_all_sentences = groupby[groupby['noise_method']==2]['sentence'].tolist()
cond1 = train_df['noise_method']=='all'
cond2 = train_df['sentence'].isin(not_all_sentences)
train_df = train_df[(cond1 & cond2)==False].reset_index(drop=True)

In [26]:
dev_df = df[df['split'] == 'dev'].drop(columns=['split']).reset_index(drop=True)
groupby = dev_df.groupby(['sentence'])[['noise_method']].count().reset_index()
not_all_sentences = groupby[groupby['noise_method']==2]['sentence'].tolist()
cond1 = dev_df['noise_method']=='all'
cond2 = dev_df['sentence'].isin(not_all_sentences)
dev_df = dev_df[(cond1 & cond2)==False].reset_index(drop=True)

In [27]:
test_df = df[df['split'] == 'test'].drop(columns=['split']).reset_index(drop=True)
groupby = test_df.groupby(['sentence'])[['noise_method']].count().reset_index()
not_all_sentences = groupby[groupby['noise_method']==2]['sentence'].tolist()
cond1 = test_df['noise_method']=='all'
cond2 = test_df['sentence'].isin(not_all_sentences)
test_df = test_df[(cond1 & cond2)==False].reset_index(drop=True)

In [28]:
train_ds = datasets.Dataset.from_pandas(train_df)
dev_ds = datasets.Dataset.from_pandas(dev_df)
test_ds = datasets.Dataset.from_pandas(test_df)

In [29]:
ds = datasets.DatasetDict({
    'train':train_ds,
    'dev': dev_ds,
    'test': test_ds
})

In [30]:
ds.push_to_hub(repo_id=f'jwengr/M-ABSA-ko-noise')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/15 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/596 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/jwengr/M-ABSA-ko-noise/commit/5f04116a97b9f306f179c77a9df6697f879d964f', commit_message='Upload dataset', commit_description='', oid='5f04116a97b9f306f179c77a9df6697f879d964f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/jwengr/M-ABSA-ko-noise', endpoint='https://huggingface.co', repo_type='dataset', repo_id='jwengr/M-ABSA-ko-noise'), pr_revision=None, pr_num=None)